In [1]:
import pandas as pd
import numpy as np
from itertools import groupby
from operator import itemgetter
import subprocess
import sys
import os
import csv
import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams['font.family'] = "serif"
%matplotlib inline

gff3Cols=["seqid","source","type","start","end","score","strand","phase","attributes"]
novelSources=['CHESS','StringTie','FANTOM']

In [2]:
# first build a dataframe of all known transcripts per
df_known_only=pd.read_csv("./chess2.02.gff",sep="\t",names=gff3Cols)
df_known_only.dropna(inplace=True,axis=0)
df_known_only.reset_index(inplace=True,drop=True)
df_known_only["start"]=df_known_only["start"].astype(int)
df_known_only["end"]=df_known_only["end"].astype(int)

# next we shall create the subset of all non-novel genes
print(set(df_known_only["source"]))
df_known_only=df_known_only[~(df_known_only['source'].isin(novelSources))].reset_index(drop=True)

# now let's isolate the protein-coding genes and their transcripts, exons and CDS

# first let's extract IDs
df_known_only["id"]=df_known_only.attributes.str.split("ID=",expand=True)[1].str.split(";",expand=True)[0]
df_known_only["parent"]=df_known_only.attributes.str.split("Parent=",expand=True)[1].str.split(";",expand=True)[0]
df_known_only["geneID"]=np.where(df_known_only["type"].isin(['transcript','exon','CDS']),df_known_only.parent.str.extract('(CHS.(\d)*)',expand=True)[0],df_known_only['id'])
# first create a tmpdf_known_only (to be removed right after) of just the genes, so we can get their coding potential
tmp=df_known_only[df_known_only["type"]=="gene"].reset_index(drop=True)
# next let's extract information about the gene type
tmp["gene_type"]=tmp.attributes.str.split("GENE_TYPE=",expand=True)[1].str.split(";",expand=True)[0].str.strip("\n")
print("set of all gene_types in CHESS",set(tmp["gene_type"]))

# now we can get a subset of all known protein_coding genes
tmp=tmp[tmp["gene_type"]=="protein_coding"].reset_index(drop=True)
print("number of known protein-coding genes is: %d"%len(tmp))
# now we just need to get geneIDs for the protein_coding sequences
setProtGenes_known_only=set(tmp["geneID"])
del tmp
# df_known_only.drop(["id","parent"],inplace=True,axis=1)
df_known_only=df_known_only[df_known_only["geneID"].isin(setProtGenes_known_only)].reset_index(drop=True)

# next we would like to test whether all transcripts in these known protein-coding genes contain a CDS

# first get a set of parent IDs for all CDSs
cdsParents_known_only=set(df_known_only[df_known_only["type"]=="CDS"]["parent"])
# now form a set of IDs for all transcripts
transIDs_known_only=set(df_known_only[df_known_only["type"]=="transcript"]["id"])
print("the number of transcripts in protein-coding known genes is: %d\nthe number of CDSs associated with transcripts in protein-coding known genes is %d"%(len(transIDs_known_only),len(cdsParents_known_only)))

# First order of business is to find out how to get ORFs out of the gffs

# one way would be to group all CDS-sequences from same transcript
# and to compile all starts and ends into format similar to tlst
# the tlst block format can then be used as a unique identifier of the intron chain as well as the orf in general
# if multiple transcripts exist for the same gene with the same CDS - those can later be merged as well.
# Otherwise a gene has isoforms coding different proteins
cdsDF=df_known_only[df_known_only['type']=='CDS'].reset_index(drop=True)

def formBlocks(series):
    return ",".join([str(x) for x in sorted(series.tolist())])

cdsGDF=cdsDF.groupby("parent").agg({'start':formBlocks,'end':formBlocks}).reset_index()
mergedDF=cdsDF[['seqid','parent','strand']].merge(cdsGDF,on='parent',how='outer',indicator=True)
assert len(mergedDF[mergedDF["_merge"]=="both"])==len(mergedDF), "ids don't match"
mergedDF.drop_duplicates(inplace=True)
mergedDF.reset_index(drop=True,inplace=True)
mergedDF.drop('_merge',inplace=True,axis=1)

mergedDF['uid']=mergedDF["start"]+"-"+mergedDF["end"]
mergedDF=mergedDF[['seqid','strand','parent','uid']]

# now we can extract gene IDs and groupby them
# we should count the total number of isoforms included
# as well as the number of unique ORFs present for that gene

# first get the gene ID
mergedDF["gID"]=mergedDF["parent"].reset_index(drop=True).str.extract('(CHS.(\d)*)',expand=True)[0]

setKnownTranscripts=set(mergedDF["parent"])
setKnownGenes=set(mergedDF["gID"])

# now let's get rid of any duplicates in this dataframe
mergedDF.drop_duplicates(['seqid','strand','gID','uid'],inplace=True)
mergedDF.reset_index(drop=True,inplace=True)

{'BestRefSeq', 'GENCODE', 'StringTie', 'HAVANA', 'ENSEMBL', 'Curated Genomic', 'Gnomon', 'RefSeq', 'CHESS', 'FANTOM'}
set of all gene_types in CHESS {'protein_coding', 'misc_RNA', 'lncRNA', 'antisense_RNA'}
number of known protein-coding genes is: 22659
the number of transcripts in protein-coding known genes is: 169965
the number of CDSs associated with transcripts in protein-coding known genes is 129847


In [3]:
# next build a dataframe of all novel transcripts and exons per known genes
df=pd.read_csv("./chess2.02.gff",sep="\t",names=gff3Cols)
df.dropna(inplace=True,axis=0)
df.reset_index(inplace=True,drop=True)
df["start"]=df["start"].astype(int)
df["end"]=df["end"].astype(int)
df["id"]=df.attributes.str.split("ID=",expand=True)[1].str.split(";",expand=True)[0]
df["parent"]=df.attributes.str.split("Parent=",expand=True)[1].str.split(";",expand=True)[0]
df["geneID"]=np.where(df["type"].isin(['transcript','exon','CDS']),df.parent.str.extract('(CHS.(\d)*)',expand=True)[0],df['id'])

df=df[(df["geneID"].isin(setKnownGenes))&(df["source"].isin(novelSources))&~((df["parent"].isin(setKnownTranscripts))|(df["id"].isin(setKnownTranscripts)))].reset_index(drop=True)

# now we can further refine the original mergedDF by removing any genes that have no novel transcripts
mergedDF=mergedDF[mergedDF['gID'].isin(set(df["geneID"]))].reset_index(drop=True)
print("the final number of known genes we are examining is: %d"%(len(set(mergedDF["gID"]))))

exonDF=df[df['type']=='exon'].reset_index(drop=True)

exonGDF=exonDF.groupby("parent").agg({'start':formBlocks,'end':formBlocks}).reset_index()
mergedExonDF=exonDF[['seqid','parent','strand']].merge(exonGDF,on='parent',how='outer',indicator=True)
assert len(mergedExonDF[mergedExonDF["_merge"]=="both"])==len(mergedExonDF), "ids don't match"
mergedExonDF.drop_duplicates(inplace=True)
mergedExonDF.reset_index(drop=True,inplace=True)
mergedExonDF.drop('_merge',inplace=True,axis=1)

mergedExonDF['uid']=mergedExonDF["start"]+"-"+mergedExonDF["end"]
mergedExonDF=mergedExonDF[['seqid','strand','parent','uid']]

# now we can extract gene IDs and groupby them
# we should count the total number of isoforms included
# as well as the number of unique ORFs present for that gene

# first get the gene ID
mergedExonDF["gID"]=mergedExonDF["parent"].reset_index(drop=True).str.extract('(CHS.(\d)*)',expand=True)[0]

# now let's get rid of any duplicates in this dataframe
mergedExonDF.drop_duplicates(['seqid','strand','gID','uid'],inplace=True)
mergedExonDF.reset_index(drop=True,inplace=True)

print("the total number of novel transcripts in known genes we are examining is: %d"%(len(set(mergedExonDF["parent"]))))

the final number of known genes we are examining is: 16496
the total number of novel transcripts in known genes we are examining is: 95974


In [4]:
# Now we create a dictionary of all exon chains of known transcripts on known genes
def groupBlocks(series):
    return series.tolist()

geneBlocks=mergedDF.groupby("gID").agg({'uid':groupBlocks}).reset_index()

knownChains=pd.Series(geneBlocks.uid.values,index=geneBlocks.gID).to_dict()
print(knownChains["CHS.39"])
mergedDF[mergedDF["gID"]=='CHS.39']

['924432,925922,930155,931039,935772,939040,939275-924948,926013,930336,931089,935896,939129,939291', '925942,930155,931039,935772-926013,930336,931089,935793', '925942,930155,931039,935772,939040,939275,941144,942136,942410,942559,943253,943698,943908-926013,930336,931089,935896,939129,939460,941306,942251,942488,943058,943377,943808,944153', '930312,931039,935772,939040,939275,941144,942136,942410,942559,943253,943698,943908-930336,931089,935896,939129,939412,941306,942251,942488,943058,943377,943808,944151', '939275,941144,942136,942410,942559,943253,943698-939460,941306,942251,942488,943058,943377,944151']


,seqid,strand,parent,uid,gID
2,chr1,+,CHS.39.3,"924432,925922,930155,931039,935772,939040,9392...",CHS.39
3,chr1,+,CHS.39.11,"925942,930155,931039,935772-926013,930336,9310...",CHS.39
4,chr1,+,CHS.39.13,"925942,930155,931039,935772,939040,939275,9411...",CHS.39
5,chr1,+,CHS.39.16,"930312,931039,935772,939040,939275,941144,9421...",CHS.39
6,chr1,+,CHS.39.21,"939275,941144,942136,942410,942559,943253,9436...",CHS.39


In [5]:
# Now we can proceed to write a comparrison function to parse these exon-intron chains
# the function is not expected to be particularly fast
# however, what I need is just to get an answer, so...

def checkCompat(orf,exons):
    # first convert the orf into lists
    tmp=orf.split("-")
    orfStarts=[int(x) for x in tmp[0].split(",")]
    orfEnds=[int(x) for x in tmp[-1].split(",")]
    
    # next we need to parse the exons in the same manner
    tmp=exons.split("-")
    exonStarts=[int(x) for x in tmp[0].split(",")]
    exonEnds=[int(x) for x in tmp[-1].split(",")]
    numExons=len(exonStarts)
    
    # now we need to compare them to one another
    # there could be three possibilities which we shall use the following return codes for:
    # 0 - ORF fits entirely within the exon
    # 1 - section of the ORF is lost without a frameshift
    # 2 - an extra section is added to the ORF without causing a framesift
    # 3 - there is a frameshift due to an extra section added to the ORF
    # 4 - there is a frameshift due to a section removed from the ORF
    # 5 - ORF is trimmed at the end - perhaps the exon should be extended???
    # 6 - potentially missing start codon - if the beginning of the ORF is trimmed
    
    assert len(orfStarts)==len(orfEnds),"wrong CDS chain passed, length don't match: "+orf
    assert len(exonStarts)==len(exonEnds),"wrong exon chain passed, length don't match: "+exons
#     assert len(orfStarts)<=len(exonStarts),"There are more segments in ORF than in the exon chain. ORF: "+orf+" ; Exons-chain: "+exons
    
    frameShift=0
    modification=0
    
    orfPos=0
    exonPos=-1
    numOrfSegments=len(orfStarts)
    for es,ee in zip(exonStarts,exonEnds):
        exonPos+=1
        curOrfStart=orfStarts[orfPos]
        curOrfEnd=orfEnds[orfPos]
        if ee<curOrfStart:
            # if this happens before the first ORF - then good
            # otherwise we have an insertion and need to check for a frameshift
            if orfPos==0:
                continue
            else:
                frameShift=((ee+1)-es)%3
                if frameShift==0:
                    modification=2
                else:
                    return 3
        elif es<=curOrfStart and ee==curOrfEnd: # ORF within exon and fits nicely - exon/CDS segment share end
            if orfPos==numOrfSegments-1: # ORF fits and is done
                return modification
            else: # ORF continues
                if exonPos==numExons-1: # trimmed based on having a shorter exon chain
                    return 5
                orfPos+=1
        elif es<=curOrfStart and ee>curOrfEnd: # could be the proper end of the ORF or a signal for modification
            if orfPos==numOrfSegments-1: # ORF fits and is done
                return modification
            else: # ORF does not match current exon chain
                return 9999999
        elif es<=curOrfStart and ee<curOrfEnd: # could be the end of the ORF, or a signal for modification
            if orfPos==numOrfSegments-1: # ORF fits, is trimmed at the end and is done
                return 5
            else: # ORF does not match current exon chain.
                return 9999999
        elif es>curOrfStart and ee==curOrfEnd: # frameshift due to exon beginning later
            if orfPos==0: # potentially missing start codon due to ORF trimming at start
                return 6
            frameShift=(es-curOrfStart)%3
            if orfPos==numOrfSegments-1:
                if frameShift==0:
                    return 1
                else:
                    return 4
            else:
                return 9999999
        elif es>curOrfStart and ee>curOrfEnd:
            frameShift=(es-curOrfStart)%3
            if orfPos==numOrfSegments-1:
                if frameShift==0:
                    return 1
                else:
                    return 4
            else:
                if frameShift==0:
                    return 9999999
                else:
                    return 4 # just abandon - not going to be any worth - it's already a frameshift
        else:
            break
                
    return ""

In [6]:
count=0
# what if we try sets
# resChain=[]
# for t in chain2:
#     resChain=resChain+list(range(t[0],t[1]))
# resOrf=[]
# for t in orf3:
#     resOrf=resOrf+list(range(t[0],t[1]))
    
# print(set(resOrf)-set(resChain))
# print(set(resChain)-set(resOrf))

def checkCompat(orf,chain):
    global count
    if count%10000==0:
        print(count)
    count+=1
    tmp=orf.split("-")
    orfStarts=[int(x) for x in tmp[0].split(",")]
    orfEnds=[int(x) for x in tmp[-1].split(",")]
    
    # next we need to parse the exons in the same manner
    tmp=chain.split("-")
    chainStarts=[int(x) for x in tmp[0].split(",")]
    chainEnds=[int(x) for x in tmp[-1].split(",")]
    
    orf=[]
    for t in zip(orfStarts,orfEnds):
        orf+=list(range(t[0],t[1]+1))
    
    chain=[]
    for t in zip(chainStarts,chainEnds):
        chain+=list(range(t[0],t[1]+1))
    
    # now remove anything in the chain that is before or after the start and end of the ORF
    chain=[x for x in chain if x>=orf[0] and x<=orf[-1]]
    
    orf=set(orf)
    chain=set(chain)
    
    # now get set differences
    orf_chain=orf-chain
    chain_orf=chain-orf
    
    if len(orf_chain)==0 and len(chain_orf)==0:
        return 0
    
    allMis=[]
    for k, g in groupby(enumerate(orf_chain), lambda ix : ix[0] - ix[1]):
        allMis.append(list(map(itemgetter(1), g)))
    for k, g in groupby(enumerate(chain_orf), lambda ix : ix[0] - ix[1]):
        allMis.append(list(map(itemgetter(1), g)))
    # now just need to figure out what kind of frameshifts we've got here
    offFrame=0 # number of extra or missing bases that do result in a frameshift
    onFrame=0 # number of extra or missing bases that do not result in a frameshift
    
    # for now this will be a very easy thing to do
    # would be better to have a method for computing how many bases are in off-frame vs on-frame regions
    for mis in allMis:
        if not len(mis)%3==0:
            return 2
    return 1
    
orf3="1,7-3,15"
chain17="1,9,15-2,9,18"
checkCompat(orf3,chain17)

# need to deal with the special case when the first base of the start codon is missing

# next apply this function to the entirety of the dataframe

def compareToAll(row): # performs comparrisons for all
    # stop if a single match is found - otherwise
    finalRes=[]
    for chain in knownChains[row['gID']]:
        res=checkCompat(chain,row['uid'])
        if res==0:
            return 0
        finalRes.append(res) # return the smallest code
        # this assumes that the smalles code is the best - sorting needs to be done in a custom way here
        # but the idea is the same
    
    return min(finalRes)

mergedExonDF["match"]=mergedExonDF.apply(lambda row: compareToAll(row),axis=1)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000


In [37]:
mergedExonDF[(mergedExonDF["match"]==1)&(mergedExonDF["match"]==0)]

,seqid,strand,parent,uid,gID,match


In [11]:
print("percent good:",len(mergedExonDF[(mergedExonDF["match"]==0)|(mergedExonDF["match"]==1)])/len(mergedExonDF))
print("percent bad:",len(mergedExonDF[mergedExonDF["match"]==2])/len(mergedExonDF))

percent good: 0.5022089315856377
percent bad: 0.4977910684143622


In [24]:
# next we need to examine the known transcripts more closely
# in order to find out the rates of 0/1/2 in there
# for each protein coding gene we should do the following
# 1. get a group of all transcripts
# 2. for each transcript
#    - compare to all other transcripts in a group
# 3. record how many transcripts per gene have a 0,1,2 when compared to the other ones

In [ ]:
# first we shall build a dataframe of all known exons to compare against known CDS sequences


In [ ]:
mergedDF.head().apply(lambda row: compareToAll(row),axis=1)

In [ ]:
# I wonder if we could use anything else here like gffcompare - unlikely
# However, we might also need to account for a missing start codon as a special code

In [ ]:
# the next logical step would be to analyze existing/known ORFs only
# to see how frequently ORFs within the same gene have frameshifts/deletions/insertions

In [ ]:
# lastly, we need to investigate novel genes and the transcripts annotated there
# to see if the distribution of frameshifts/insertions/deletions is similar
# to that observed for the known transcripts inside known genes

# all in all these steps should answer the question completely

one other important consideration is (not critical at the moment, but will be necessary for the final results)

At the moment we are removing duplicates of any transcripts which appear to have identical intron-exon chain,
whihc is done to avoid making comparisons multiple times

What we should be doing instead is:
1. grouping them by the same ['seqid','strand','blocks']
2. keeping track of the number that went into the same group
3. keeping only one single representative for the group, thus effectively performing the same function as the drop_duplicates()